Basic Usage of ExeTera
1, Loading the Exetera library in R

In [1]:
# Set the system environment of the path contains python binary
Sys.setenv(RETICULATE_PYTHON_ENV="/home/jd21/miniconda3/bin/")

In [2]:
library(reticulate)
library(devtools)
#Once both library loaded, use py_config() to check which python evnironment is used. If not the one 
#has ExeTera installed, call use_python() to change.
py_config()

Loading required package: usethis



python:         /home/jd21/miniconda3/bin/python
libpython:      /home/jd21/miniconda3/lib/libpython3.8.so
pythonhome:     /home/jd21/miniconda3:/home/jd21/miniconda3
version:        3.8.5 (default, Sep  4 2020, 07:30:14)  [GCC 7.3.0]
numpy:          /home/jd21/miniconda3/lib/python3.8/site-packages/numpy
numpy_version:  1.20.3

NOTE: Python version was forced by RETICULATE_PYTHON_ENV

In [3]:
# Now load the ExeTera library; if not installed, use pip install exetera & pip install exetera-covid
EXETERA_R_WRAPPER = "/home/jd21/codes/exetera"
load_all(EXETERA_R_WRAPPER)
exetera  # Now use can use exetera$ to call python class and methods

ℹ Loading exetera



Module(exetera)

2, Processing a dataset using Exetara

In [4]:
# First, open an exetera session
session = Session()  

In [5]:
#  Note in python, the Exetera session is open via with statement. Without the same
# statement in R, you need to call session$close() manually in the end of the script to make sure the
# files are closed properly.
# session$close()

In [6]:
#Open the example dataset
src = session$open_dataset('example.hdf5', 'r', 'src')  # filename, mode (r, r+, or w) and name in session
dst = session$open_dataset('output.hdf5', 'w', 'dst') # Store the result
result = dst$create_dataframe('results')
# List the name of dataframes in the dataset
ds.keys(src)

[1] "df"  "df2"

In [7]:
# Get the Exetera dataframe object
df = src['df']
# List the fields in dataframe
df.keys(df)

[1] "datestamp"    "fixed_string" "num"

# change df.keys(df) to names(df) or colnames()

In [8]:
# Get the field object
field = df['num']
# Get the field content
fld.data(field)
# Partial content
fld.data(field,0:3) # note the python array index start from zero

[1] 0 1 2 3 4 5 6 7 8 9

[1] 0 1 2 3

In [9]:
# Some ExeTera Field APIs
# Spans
field$get_spans()

# Filter
filter = ifelse(fld.data(field)<5, TRUE, FALSE)
fld.data(field$apply_filter(filter)) # get applied content

# Create a new field
num2 = field$create_like(result, 'num2')
field$apply_filter(filter, target=num2)  # write result to the new field

# Re-indexing
order_index = sort(fld.data(field), decreasing=TRUE)
fld.data(field$apply_index(order_index))

# Create a new field
num3 = field$create_like(result, 'num3')
field$apply_index(order_index, target=num3)  # write result to the new field

[1]  0  1  2  3  4  5  6  7  8  9 10

[1] 0 1 2 3 4

<exetera.core.fields.NumericField>

[1] 9 8 7 6 5 4 3 2 1 0

<exetera.core.fields.NumericField>

In [10]:
# Some ExeTera DataFrame APIs
df = src['df2']
field = df['num']

# Filter
result2 = dst$create_dataframe('result2')
filter = ifelse(fld.data(field)<5, TRUE, FALSE)
df$apply_filter(filter, ddf=result2)  # destructive if the ddf= is not specified
fld.data(result2['num'])

# # Re-indexing
result3 = dst$create_dataframe('result3')
order_index = sort(fld.data(field), decreasing=TRUE)
df$apply_index(order_index, ddf=result3)
fld.data(result3['num'])

# Merge
result4 = dst$create_dataframe('result4')
exetera$core$dataframe$merge(left=src['df'], right=src['df2'], dest=result4, left_on='num', right_on='num')
df.keys(result4)
fld.data(result4['num_l'])

<exetera.core.dataframe.HDF5DataFrame>

[1] 0 1 2 3 4

<exetera.core.dataframe.HDF5DataFrame>

[1] 9 8 7 6 5 4 3 2 1 0

[1] "datestamp"      "fixed_string_l" "num_l"          "fixed_string_r"
[5] "num_r"

[1] 0 1 2 3 4 5 6 7 8 9

In [11]:
session$close()